# Reto 4 - Concesiones de préstamo

**Introducción al Problema**

En el ámbito financiero, las instituciones crediticias enfrentan el desafío de evaluar la solvencia de los solicitantes de préstamos para minimizar el riesgo de incumplimiento. Este ejercicio se centra en aplicar algoritmos de clasificación para predecir el estado de aprobación de un préstamo basándose en las características demográficas, financieras y crediticias de los solicitantes. Al analizar este conjunto de datos, los alumnos podrán profundizar en técnicas de preprocesamiento, análisis exploratorio y modelado predictivo.

**Descripción del Conjunto de Datos**

El conjunto de datos proporciona información detallada sobre solicitantes de préstamos, incluyendo variables que describen sus características personales, financieras y su historial crediticio. A continuación, se presenta una descripción detallada de cada una de las variables:

1. **person_age**: Edad del solicitante del préstamo (en años).
   - *Tipo de variable*: Numérica continua.
   - *Análisis*: La edad puede influir en la estabilidad financiera y en el riesgo percibido por la entidad crediticia.

2. **person_gender**: Género del solicitante.
   - *Tipo de variable*: Categórica nominal ('male', 'female').
   - *Análisis*: Puede utilizarse para estudios demográficos, aunque debe manejarse con cuidado para evitar sesgos discriminatorios.

3. **person_education**: Nivel educativo alcanzado por el solicitante.
   - *Tipo de variable*: Categórica ordinal ('High School', 'Associate', 'Bachelor', 'Master', etc.).
   - *Análisis*: Un mayor nivel educativo podría correlacionarse con mejores oportunidades laborales y mayor capacidad de pago.

4. **person_income**: Ingresos anuales del solicitante (en dólares).
   - *Tipo de variable*: Numérica continua.
   - *Análisis*: Indicador directo de la capacidad financiera para asumir y pagar el préstamo.

5. **person_emp_exp**: Años de experiencia laboral del solicitante.
   - *Tipo de variable*: Numérica discreta.
   - *Análisis*: Refleja la estabilidad laboral y potencialmente la seguridad financiera del individuo.

6. **person_home_ownership**: Situación de vivienda del solicitante.
   - *Tipo de variable*: Categórica nominal ('RENT', 'OWN', 'MORTGAGE').
   - *Análisis*: Puede indicar estabilidad financiera; por ejemplo, poseer una vivienda puede servir como garantía.

7. **loan_amnt**: Monto del préstamo solicitado (en dólares).
   - *Tipo de variable*: Numérica continua.
   - *Análisis*: Cantidad de dinero que el solicitante desea obtener, importante para evaluar el riesgo crediticio.

8. **loan_intent**: Propósito del préstamo.
   - *Tipo de variable*: Categórica nominal ('PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT', etc.).
   - *Análisis*: El motivo del préstamo puede afectar el riesgo; por ejemplo, préstamos para negocios pueden ser más riesgosos que para educación.

9. **loan_int_rate**: Tasa de interés anual del préstamo (en porcentaje).
   - *Tipo de variable*: Numérica continua.
   - *Análisis*: Refleja el costo del préstamo para el solicitante; tasas más altas pueden indicar mayor riesgo percibido.

10. **loan_percent_income**: Porcentaje del ingreso anual que representa el pago del préstamo.
    - *Tipo de variable*: Numérica continua.
    - *Análisis*: Proporciona una medida de la carga financiera que el préstamo supondrá para el solicitante.

11. **cb_person_cred_hist_length**: Duración del historial crediticio del solicitante (en años).
    - *Tipo de variable*: Numérica discreta.
    - *Análisis*: Historiales más largos pueden proporcionar más información sobre el comportamiento crediticio del solicitante.

12. **credit_score**: Puntaje de crédito del solicitante.
    - *Tipo de variable*: Numérica continua.
    - *Análisis*: Indicador clave de la solvencia crediticia; puntajes más altos suelen asociarse con menor riesgo de incumplimiento.

13. **previous_loan_defaults_on_file**: Historial de incumplimientos de préstamos anteriores.
    - *Tipo de variable*: Categórica nominal ('Yes', 'No').
    - *Análisis*: Un historial de incumplimientos aumenta significativamente el riesgo percibido.

14. **loan_status**: Estado del préstamo.
    - *Tipo de variable*: Categórica binaria (1: aprobado, 0: rechazado).
    - *Análisis*: Es la variable objetivo que se pretende predecir mediante algoritmos de clasificación.

# Instalación de la librería del curso
En el caso de que estés trabajando desde Google Colab o Kaggle, será necesario instalar la librería `blackops`, descomentando (eliminando el asterisco #) y ejecutando la siguiente celda:

In [ ]:
# !pip install git+https://github.com/donielix/esic-bigdata-iv-blackops.git > /dev/null

# Importación de paquetes necesarios
Esta celda no debería modificarse. Estos son los únicos paquetes y funciones necesarias y no hay por qué añadir más. Simplemente ejecuta la celda

In [ ]:
from blackops.utils.catalog import read_table
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from catboost import CatBoostClassifier
import plotly.express as px
from sklearn.metrics import f1_score, log_loss, precision_score, recall_score

# Apartado 1 - Lectura de Datos
Utiliza la función `read_table` para leer la tabla denominada `"loan_data"`. Recuerda que debes especificar también el argumento `config_share_path`, que debe apuntar a la ruta exacta en la que tienes ubicado el fichero `config.share`, que contiene los permisos necesarios para leer la tabla deseada.

Después de leer la tabla, printea por pantalla el siguiente mensaje: `"La tabla contiene N registros"`, siendo N el número de registros que contiene la tabla recién cargada

In [ ]:
# INSERTA CÓDIGO AQUÍ

Una vez consultados los registros que tiene, dado que se trata de un volúmen pequeño, vamos a convertir el DataFrame de PySpark a un DataFrame de Pandas. Para ello, hay que hacer uso del método `.toPandas()`. Almacena el resultado en una variable llamada `df`.

In [ ]:
# INSERTA CÓDIGO AQUÍ

A partir de aquí, se asumirá que la variable `df` contiene un DataFrame de pandas. Compruébalo simplemente ejecutando la siguiente celda (no la modifiques, sólo ejecútala)

In [ ]:
isinstance(df, pd.DataFrame)

Si la celda anterior te devuelve `False`, o si te arroja un error, debes revisar de nuevo que hayas leído la tabla correctamente y que hayas transformado el DataFrame resultante a un DataFrame de pandas, almacenado en la variable `df`.

Ahora, debes obtener una descripción completa del DataFrame, incluyendo todos estos valores por cada columna:
* count
* unique
* top
* freq
* mean
* std
* min
* 25%
* 50%
* 75%
* max

Para ello, utiliza el método `describe` con el argumento `include="all"` para mostrar toda esa info.

In [ ]:
# INSERTA CÓDIGO AQUÍ

Tras mostrar la descripción de los datos, responde a las siguientes pregunta:
* ¿Cuál es el propósito de préstamo más recurrente? ¿con qué frecuencia aparece?:
* ¿Existe algún valor nulo en el dataset?
* ¿Cuál es la media de edad del solicitante?
* ¿Observas algún valor anómalo en los datos?

Ahora, utiliza la función `px.imshow` que viene dentro de la librería de plotly para mostrar un mapa de calor con la correlación de las variables numéricas en el DataFrame. Esta correlación la puedes calcular haciendo uso del método `.corr`, empleando el argumento `numeric_only=True` para que se aplique únicamente sobre variables de tipo número.

In [ ]:
# INSERTA CÓDIGO AQUÍ

Analiza el gráfico y observa como hay dos variables que están altamente correlacionadas linealmente, con un coeficiente superior a 0.95. No queremos utilizar ambas a la hora de modelizar ya que introduce redundancias en el modelo. Piensa cuál de ellas podrías eliminar, y elimina dicha columna del DataFrame utilizando el método `.drop` (recuerda asignar el resultado de nuevo a la variable `df`)

In [ ]:
# INSERTA CÓDIGO AQUÍ

# Apartado 2 - Preparación de los datos
En primer lugar define las variables `X` e `y`, que contendrán las variables precictoras y la variable target, respectivamente

In [ ]:
# INSERTA CÓDIGO AQUÍ

Ahora vamos a comprobar si el target está balanceado. Para ello, haz uso del método `.value_counts()` aplicado a la variable target

In [ ]:
# INSERTA CÓDIGO AQUÍ

Después, haz uso de la función `train_test_split` para definir los conjuntos de entrenamiento y de test. Consulta la documentación de dicha función para ver ejemplos de su uso, y sigue la misma convención para nombrar a las variables resultantes, es decir:

`X_train, X_test, y_train, y_test = train_test_split(...)`

In [ ]:
# INSERTA CÓDIGO AQUÍ

Printea el siguiente mensaje: `"El conjunto de entrenamiento contiene N registros"`, siendo N el número de registros que contiene el dataset de entrenamiento (`X_train`)

In [ ]:
# INSERTA CÓDIGO AQUÍ

# Apartado 3 - Entrenamiento del modelo
Ahora debes entrenar el modelo `CatBoostClassifier` sobre los datos de entrenamiento (`X_train`, `y_train`). El modelo CatBoost no tiene problemas con las variables de tipo categórico (no hace falta hace One Hot Encoding); debes especificar cuáles son dichas variables en su argumento `cat_features`.

In [ ]:
# INSERTA CÓDIGO AQUÍ

Una vez entrenado el modelo, calcula las siguientes métricas sobre el conjunto de testing: `f1_score`, `precision_score` y `recall_score`. Recuerda que todas estas métricas aceptan esencialmente dos argumentos:
* `y_pred`: las predicciones del modelo, es decir, el resultado de aplicar el método `.predict` sobre el conjunto de test (`X_test`).
* `y_true`: las etiquetas reales del conjunto de test (`y_test`).

In [ ]:
# INSERTA CÓDIGO AQUÍ

Con los resultados de estas métricas, responde a las siguientes preguntas:
* ¿De las veces que el modelo predice que se otorga un préstamo, qué porcentaje realmente se otorga finalmente?
* ¿De todos los préstamos otorgados, qué porcentaje de los mismos es correctamente clasificado por el modelo?

Calcula la importancia de las características con el atributo `feature_importances_` del clasificador. ¿Cuál es, con diferencia, la variable más importante a la hora de determinar si se le concede el préstamo al solicitante?

In [ ]:
# INSERTA CÓDIGO AQUÍ